In [4]:
from PIL import Image
import os
import pandas as pd

In [5]:
imgs_path = "../expW/origin"
labels_path = "../expW/label.csv"
new_imgs_path = "../expW/origin_cleaned"
new_labels_path = "../expW/new_label.csv"
os.makedirs(new_imgs_path, exist_ok=True)

In [6]:
df_labels = pd.read_csv(labels_path)
new_label_entries = []
image_counter = 1

for _, row in df_labels.iterrows():
    orig_img_name = row['image_name']
    orig_img_path = os.path.join(imgs_path, orig_img_name)
    
    if not os.path.exists(orig_img_path):
        # Skip if original image missing
        continue
    
    img = Image.open(orig_img_path).convert("RGB")
    
    # Extract bounding box
    left, top = int(row['bbox_left']), int(row['bbox_top'])
    right, bottom = int(row['bbox_right']), int(row['bbox_bottom'])
    
    # Crop and resize
    cropped_face = img.crop((left, top, right, bottom)).resize((224, 224), resample=Image.LANCZOS)
    
    # Save with new name
    new_img_name = f"Image_{image_counter}.jpg"
    cropped_face.save(os.path.join(new_imgs_path, new_img_name))
    
    # Save label info without extension
    new_label_entries.append({
        'image_name': f"Image_{image_counter}",
        'expression_label': int(row['expression_label'])
    })
    
    image_counter += 1

# Save new label CSV
new_labels_df = pd.DataFrame(new_label_entries)
new_labels_df.to_csv(new_labels_path, index=False)